# CIFAR-10 Benchmarking

In [ ]:
from torcheval.metrics.functional import multiclass_accuracy
from sklearn.metrics import accuracy_score
from copy import deepcopy

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import torchvision
import random
import torch
import time
import os

In [ ]:
SEED = 42
random.seed(SEED)
_ = torch.manual_seed(SEED)
_ = np.random.seed(SEED)
_ = os.environ["PYTHONHASHSEED"] = str(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

In [ ]:
DEVICE = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else (
        torch.device("mps")
        if torch.backends.mps.is_available()
        else torch.device("cpu")
    )
)
MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]
VIZ_FOLDER = "visualizations"
WEIGHTS_FOLDER = "weights"
METRICS_FOLDER = "metrics"
print(f"Hardware Accelerator - {DEVICE.type}")

In [ ]:
os.makedirs(VIZ_FOLDER, exist_ok=True)
os.makedirs(WEIGHTS_FOLDER, exist_ok=True)
os.makedirs(METRICS_FOLDER, exist_ok=True)

In [ ]:
denorm = lambda x: torch.clamp(
    x * torch.tensor(STD).view(-1, 1, 1) + torch.tensor(MEAN).view(-1, 1, 1),
    min=0.0,
    max=1.0,
)

In [ ]:
train_data_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomHorizontalFlip(0.5),
        torchvision.transforms.RandomAutocontrast(0.5),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(MEAN, STD),
    ]
)
test_data_transforms = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize(MEAN, STD)]
)
train_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=train_data_transforms
)
test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=test_data_transforms
)
classes = train_dataset.classes
print(f"Size of train data - {len(train_dataset)}")
print(f"Size of test data - {len(test_dataset)}")
print(f"Classes in dataset - {', '.join(train_dataset.classes)}")

In [ ]:
plt.figure(figsize=(7, 7))
for i in range(9):
    image = denorm(train_dataset[i][0]).permute(1, 2, 0).numpy()
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(f"Label: {classes[train_dataset[i][1]]}")
    plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize=(7, 7))
for i in range(9):
    image = denorm(test_dataset[i][0]).permute(1, 2, 0).numpy()
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(f"Label: {classes[test_dataset[i][1]]}")
    plt.axis("off")
plt.show()

In [ ]:
class DepthwiseSeparableConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = torch.nn.Conv2d(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            groups=in_channels,
            bias=False,
        )
        self.pointwise = torch.nn.Conv2d(
            in_channels, out_channels, kernel_size=1, stride=1, bias=False
        )
        self.bn = torch.nn.BatchNorm2d(out_channels)
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [ ]:
class VolumePreservingCNN(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(VolumePreservingCNN, self).__init__()
        self.block1 = DepthwiseSeparableConv(in_channels=3, out_channels=12)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.block2 = DepthwiseSeparableConv(in_channels=12, out_channels=48)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.block3 = DepthwiseSeparableConv(in_channels=48, out_channels=192)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.block4 = DepthwiseSeparableConv(in_channels=192, out_channels=768)
        self.pool4 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.block5 = DepthwiseSeparableConv(in_channels=768, out_channels=3072)
        self.pool5 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.global_pool = torch.nn.AdaptiveMaxPool2d((1, 1))
        self.fc = torch.nn.Linear(3072, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = self.block3(x)
        x = self.pool3(x)
        x = self.block4(x)
        x = self.pool4(x)
        x = self.block5(x)
        x = self.pool5(x)
        x = self.global_pool(x)
        x = torch.flatten(x, start_dim=1)
        out = self.fc(x)
        return out